In [9]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

# 东京 Apple Store 门店列表（ID 和名称）
tokyo_stores = [
    {"id": "R079", "name": "Apple Marunouchi"},
    {"id": "R119", "name": "Apple Ginza"},
    {"id": "R128", "name": "Apple Shibuya"},
    {"id": "R224", "name": "Apple Omotesando"},
    {"id": "R718", "name": "Apple Omotesando"},
    {"id": "R710", "name": "Apple Shinjuku"}
]

# API 基础 URL
base_url = "https://www.apple.com/jp/shop/fulfillment-messages"
init_url = "https://www.apple.com/jp/"
# SKU 参数（可替换为 'MG864J/A' 如果需要）
sku = "MFY94J/A"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Referer': 'https://www.apple.com/jp/shop/buy-iphone/iphone-17',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Connection': 'keep-alive',
    'Origin': 'https://www.apple.com'
}
# 创建 Session 并添加 retries
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504, 541])
session.mount('https://', HTTPAdapter(max_retries=retries))

# 先访问初始化页面以设置 cookie
try:
    init_response = session.get(init_url, headers=headers, timeout=10)
    print(f"初始化页面状态码: {init_response.status_code}")
    print(f"设置的 Cookies: {session.cookies.get_dict()}")
except Exception as e:
    print(f"初始化失败: {str(e)}")

# 循环查询每个门店
for store in tokyo_stores:
    params = {
        "fae": "true",
        "pl": "true",
        "mts.0": "regular",
        "mts.1": "compact",
        "cppart": "UNLOCKED_JP",
        "parts.0": sku,
        "searchNearby": "true",
        "store": store["id"]
    }

    try:
        response = session.get(base_url, params=params, headers=headers, timeout=10)
        print(f"查询 {store['name']} 状态码: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            stores = data.get('body', {}).get('content', {}).get('pickupMessage', {}).get('stores', [])
            if stores:
                target_store = next((s for s in stores if s.get('storeNumber') == store["id"]), stores[0])
                parts_availability = target_store.get('partsAvailability', {}).get(sku, {})
                availability = parts_availability.get('pickupDisplay', '未知')
                pickup_time = parts_availability.get('pickupSearchQuote', '未知')
                buyable = parts_availability.get('isBuyable', '未知')
            else:
                availability = '无门店数据'
                pickup_time = '未知'
                buyable = '未知'

            print(f"门店: {store['name']} (ID: {store['id']})")
            print(f"可用性: {availability}")
            print(f"取货时间估计: {pickup_time}")
            print(f"可购买: {buyable}")
            print("---")
        else:
            print(f"错误详情: {response.text[:200]}...")
    except Exception as e:
        print(f"查询 {store['name']} 异常: {str(e)}")

    time.sleep(2)  # 延迟避免限制

初始化页面状态码: 200
设置的 Cookies: {'geo': 'JP'}
查询 Apple Marunouchi 异常: HTTPSConnectionPool(host='www.apple.com', port=443): Max retries exceeded with url: /jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true&store=R079 (Caused by ResponseError('too many 541 error responses'))


KeyboardInterrupt: 